<a href="https://colab.research.google.com/github/rohanchidrewar05/smart_cap/blob/master/Convert_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install face_recognition

In [0]:
import numpy as np
import os
import face_recognition
import cv2
import pickle
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def change_names(data_path):
    print(data_path)
    files = os.listdir(data_path)
    print(files)
    for i in range(len(files)):
      path = os.path.join(data_path,files[i])
      files1 = os.listdir(path)
      print(len(files1))
      for j in range(len(files1)):
        #print(os.path.join(path,files1[j]))
        prev_name = files1[j]
        new_name = "%#04d" %(j,)
        print(prev_name,new_name)
        os.rename(os.path.join(path,prev_name),os.path.join(path,new_name))
    print("Dataset renaming is completed")

In [0]:
def extract_faces(data_path,data_save_path,test_data_save_path):
    X = []
    Y = []
    X_test = []
    Y_test = []
    print("Loading data ....")
    print(files)
    temp = 0                         
    for i in range(len(files)):
      path = os.path.join(data_path,files[i])
      files1 = os.listdir(path)
      files1.sort()
      print(files1)
      for j in range(len(files1)):
        image = cv2.imread(os.path.join(path,files1[j]))
        face_bounding_boxes = face_recognition.face_locations(image)
        check = False
        
        if(len(face_bounding_boxes) == 1):
          check = True
          encoding = face_recognition.face_encodings(image,known_face_locations = face_bounding_boxes)[0]
        
        elif(len(face_bounding_boxes) > 1):
          check = True
          max_area = 0
          max_loc = 0
          for i in range(len(face_bounding_boxes)):
            box = face_bounding_boxes[i] 
            area = (box[0]-box[2])*(box[3]-box[1])
            if(max_area < area):
              area = max_area
              max_box = i
          templist = []
          templist.append(face_bounding_boxes[max_box])
          encoding = face_recognition.face_encodings(image,known_face_locations = templist)[0]
        
        if(j < 0.8 * len(files1)):
          if(check):
            X.append(encoding)
            Y.append(i)

        else:
          if(check):
            X_test.append(encoding)
            Y_test.append(i)
            
    print("Data loading completed.")
    print("Saving the data...")
    if data_save_path is not None:
        with open(os.path.join(data_save_path,"embedding.txt"), 'wb') as f:
            pickle.dump(X, f)
        with open(os.path.join(data_save_path,"label.txt"), 'wb') as f:
            pickle.dump(Y, f)
        
    if test_data_save_path is not None:
        with open(os.path.join(test_data_save_path,"test_embedding.txt"), 'wb') as f:
            pickle.dump(X_test, f)
        with open(os.path.join(test_data_save_path,"test_label.txt"), 'wb') as f:
            pickle.dump(Y_test, f)
    print("Saving data is completed.")

In [0]:
if __name__ == '__main__':
    data_path = "/content/drive/My Drive/Colab Notebooks/Dataset"
    data_save_path = "/content/drive/My Drive/Colab Notebooks/Converted_data"
    test_data_save_path = "/content/drive/My Drive/Colab Notebooks/test_converted_data"
    print("Loading dataset from : ", data_path)
    files = os.listdir(data_path)
    change_names(data_path)
    extract_faces(data_path,data_save_path,test_data_save_path)